Загружаем картинку

In [ ]:
import Images, LinearAlgebra

fli = Images.load("relaxing_fly.png")

flm = convert(Array{Float16,2}, fli) * 2.0 .- 1.0
BASIS_D1 = size(flm)[1]

fli

Создаём базис — тригонометрический или случайный двух видов

Можно бы было [красиво и с кнопочками](https://github.com/JuliaGizmos/Interact.jl#usage), но

```
@manipulate for basis_mode = Dict("Тригонометрический" => 0, "Случайный" => 1, "Случайный с частотами" => 2)
    ... // fails for UNIX non-root users right now
end
```

In [ ]:
# Settings

@enum Basis begin
    trigonometric = 0
    randomized = 1
end

basis_type = trigonometric

rand_basis_size = 16384
rand_freqdown = true
rand_fuzzy = false
;

In [ ]:
function gen_trig_basis()
    freqs = [hn * 2 * pi / BASIS_D1 for hn in 1:BASIS_D1]
    phases = [0.0, pi/2.0]
    directions = [ [cos(a) sin(a)] for a in 0:pi/16:pi/2 ]

    [
        [
            sin(p + LinearAlgebra.dot([x y], d) * f)
            for x in 1:BASIS_D1, y in 1:BASIS_D1
        ]
        for f in freqs, p in phases, d in directions
    ]
end

function roundscale(v::Integer, ss::Integer, ts::Integer)::Integer
    Int(round((v - 1) * (ts - 1) / (ss - 1) + 1))  # arrr those 1:sizeof indices...
end

function scale2Dnearest(arr2d, th, tw)
    sh, sw = size(arr2d)
    if sh == th && sw == tw
        arr2d
    else
        [
            arr2d[roundscale(tr, th, sh), roundscale(tc, tw, sw)]
            for tc in 1:tw, tr in 1:th
        ]
    end
end

function gen_random_basis(lowerfreqs)
    bbsz = if lowerfreqs
        (()-> rand(2:BASIS_D1))
    else
        (()-> BASIS_D1)
    end
    
    bpixfun = if rand_fuzzy
        (()->rand() * 2.0 - 1.0)
    else
        (()->[Float16(-1.0) Float16(1.0)][rand(1:end)])
    end

    [
        scale2Dnearest([
            bpixfun()
            for x in 1:bbsz(), y in 1:bbsz()
        ], BASIS_D1, BASIS_D1) for _ in 1:rand_basis_size
    ]
end

basis = if basis_type == trigonometric
    gen_trig_basis()
elseif basis_type == randomized
    gen_random_basis(rand_freqdown)
else
    []
end
;    

In [ ]:
print(length(basis))
Images.Gray.((basis[10] .+ 1.0) / 2.0)

В качестве «координат» берём скалярное произведение

In [ ]:
coeffs = (LinearAlgebra.dot(b, flm) for b in basis);

In [ ]:
rflm = sum(c * b for (c, b) in zip(coeffs, basis));

Нормируем результат

In [ ]:
zrflm = rflm - ones(BASIS_D1, BASIS_D1) * minimum(rflm)
nrflm =  zrflm / maximum(zrflm)
;

In [ ]:
rfli = Images.Gray.(nrflm)